In [ ]:
!pip install sentence-transformers
!pip install yacs
!pip install SentencePiece
!pip install transformers datasets
!pip install sentence_transformers
!pip install gdown
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 5.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
!curl -L -o 'data_QA.csv' 'https://drive.usercontent.google.com/download?id=<your id>&export=download&authuser=1&confirm=t'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.9M  100 14.9M    0     0  6911k      0  0:00:02  0:00:02 --:--:-- 6909k


In [3]:
import random
import pandas as pd
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer, InputExample, SentencesDataset, losses, evaluation
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers import LoggingHandler
import logging
import os

In [4]:
df = pd.read_csv('data_QA.csv')
df=df.dropna()
train_df, valid_df = train_test_split(df, test_size=0.1, random_state=42)

def create_examples(dataframe):
    examples = []
    questions = dataframe["Question"].tolist()
    answers = dataframe["Answer"].tolist()

    # Tạo cặp có liên quan (positive pairs)
    for _, row in dataframe.iterrows():
        ques = row["Question"]
        ans = row["Answer"]
        examples.append(InputExample(texts=[str(ques), str(ans)], label=1.))

    # Tạo cặp không liên quan (negative pairs)
    for _ in range(len(dataframe)):
        ques = random.choice(questions)
        ans = random.choice(answers)
        while dataframe[(dataframe["Question"] == ques) & (dataframe["Answer"] == ans)].shape[0] > 0:
            ques = random.choice(questions)
            ans = random.choice(answers)
        examples.append(InputExample(texts=[str(ques), str(ans)], label=0.))

    return examples

# Tạo danh sách InputExample từ DataFrame cho train và validation
train_examples = create_examples(train_df)
valid_examples = create_examples(valid_df)

print(f"Train examples: {len(train_examples)}")
print(f"Validation examples: {len(valid_examples)}")

Train examples: 2000
Validation examples: 400


In [5]:
df['Question_Length'] = df['Question'].apply(lambda x: len(x.split()))
df['Answer_Length'] = df['Answer'].apply(lambda x: len(x.split()))
question_length_stats = df['Question_Length'].describe()
answer_length_stats = df['Answer_Length'].describe()
answer_length_stats

count    12772.000000
mean       185.138819
std        157.992859
min          2.000000
25%         83.000000
50%        140.000000
75%        235.000000
max       2298.000000
Name: Answer_Length, dtype: float64

In [6]:
model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
model = SentenceTransformer(model_name)
model.max_seq_length = 300

# Tạo SentencesDataset và DataLoader cho tập huấn luyện
train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, batch_size=32, drop_last=True, shuffle=True)

# Tạo SentencesDataset và DataLoader cho tập kiểm tra
valid_dataset = SentencesDataset(valid_examples, model)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, drop_last=False, shuffle=False)

# Chọn hàm mất mát CosineSimilarityLoss
train_loss = losses.CosineSimilarityLoss(model=model)

# Đường dẫn lưu mô hình
model_save_path = 'checkpoint'

# Tạo evaluator cho tập kiểm tra
evaluator = evaluation.EmbeddingSimilarityEvaluator(
    sentences1=[example.texts[0] for example in valid_examples],
    sentences2=[example.texts[1] for example in valid_examples],
    scores=[example.label for example in valid_examples],
    batch_size=32
)

def eval_callback(score, epoch, steps):
    print(f"Epoch: {epoch}, Steps: {steps}, Score: {score}")

logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO, handlers=[LoggingHandler()])

# Huấn luyện mô hình
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    evaluator=evaluator,
    epochs=2,
    warmup_steps=int(len(train_dataloader) * 0.1),
    evaluation_steps=300,
    output_path=model_save_path,
    callback=eval_callback,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

Epoch: 0, Steps: -1, Score: 0.8641661496897396


Iteration:   0%|          | 0/62 [00:00<?, ?it/s]

Epoch: 1, Steps: -1, Score: 0.8649455749888534
